# 데이터 주소 가져오기 

In [ ]:
# 1. 좌표 변환 하기 
# 2. 좌표 구분 컬럼 생성 
# 3. 중복 좌표 삭제 > url 생성 
# 4. API를 통해 주소 가져오기 
# 5. raw_data 와 merge 
# 6. pkl 저장하기 

In [ ]:
############# data_6월 ############## 

In [2]:
# 데이터 가져오기

raw_data_6 = pd.read_csv('../../1. raw_data/KRI-DAC_Jeju_data6.txt', sep=',')
raw_data_6.tail()

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y
281891,281892,281892,202006,제주특별자치도,제주시,영세,일반한식,x시,175000,0,2,0,950274.0385,1.500908e+06
281892,281893,281893,202006,제주특별자치도,제주시,영세,일반한식,x시,600000,0,2,0,950266.5976,1.502458e+06
281893,281894,281894,202006,제주특별자치도,제주시,영세,편의점,x시,6300,0,1,0,950266.5976,1.502458e+06
281894,281895,281895,202006,제주특별자치도,제주시,중소,제과점,x시,15000,0,2,0,950524.6587,1.500760e+06
281895,281896,281896,202006,제주특별자치도,제주시,중소1,중국음식,x시,15000,0,1,0,950574.6388,1.500760e+06


In [6]:
# 1. 좌표 변환 하기 

df_6 = raw_data_6.copy()

from pyproj import Proj, transform
# ITRF좌표계
proj_ITRF = Proj(init='epsg:5179')
# WGS 좌표계 
proj_WGS84 = Proj(init='epsg:4326')


df_6['lon'], df_6['lat'] = transform(proj_ITRF, proj_WGS84, df_6['POINT_X'], df_6['POINT_Y'])

df_6.tail(3)

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lon,lat
281893,281894,281894,202006,제주특별자치도,제주시,영세,편의점,x시,6300,0,1,0,950266.5976,1.502458e+06,126.964496,33.512854
281894,281895,281895,202006,제주특별자치도,제주시,중소,제과점,x시,15000,0,2,0,950524.6587,1.500760e+06,126.967369,33.497551
281895,281896,281896,202006,제주특별자치도,제주시,중소1,중국음식,x시,15000,0,1,0,950574.6388,1.500760e+06,126.967907,33.497555


In [8]:
# 2. 좌표 구분 컬럼 생성 

df_6['lon'] = df_6['lon'].astype(str)
df_6['lat'] = df_6['lat'].astype(str)
df_6['lon_lat'] = df_6['lon']+'_'+df_6['lat']
df_6['lon_lat'].value_counts()

126.52427851055405_33.51290574138535     410
126.52880376641139_33.499886709171996    308
126.52935695016222_33.49899169591379     305
126.47793511858133_33.48574932701181     295
126.57376368512601_33.51438638164462     268
                                        ... 
126.50209229609702_33.48874811600121       1
126.70977311456376_33.4879264711372        1
126.64109526921277_33.54489489629949       1
126.45976766858116_33.47876332744271       1
126.48927703444896_33.48318327512829       1
Name: lon_lat, Length: 13909, dtype: int64

In [9]:
# 3. 중복 좌표 삭제 > url 생성 

df_uniq = df_6.drop_duplicates('lon_lat')
df_uniq.value_counts('lon_lat')

lon_lat
126.96951006044938_33.49847009919258     1
126.48496931110616_33.452477320750525    1
126.48522506380449_33.468708740463306    1
126.4851545536666_33.472764947994555     1
126.48510753973808_33.47546908389631     1
                                        ..
126.55745873241018_33.263554759877884    1
126.55745133042375_33.26400547881826     1
126.55744392829351_33.26445619862645     1
126.55743652602919_33.26490691840057     1
126.16370558240443_33.33544861735468     1
Length: 13909, dtype: int64

In [10]:
# url 생성

df_uniq['url1'] = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=' + df_uniq['lon']
df_uniq['url2'] = '&y=' + df_uniq['lat']
df_uniq['url'] = df_uniq['url1'] + df_uniq['url2']


df_uniq.drop(columns=['url1','url2'],axis=1,inplace=True)
df_uniq = df_uniq.reset_index()
df_uniq.head(5)

,index,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lon,lat,lon_lat,url
0,0,1,1,202006,제주특별자치도,제주시,영세,일반한식,00시,502000,0,10,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,https://dapi.kakao.com/v2/local/geo/coord2regi...
1,1,2,2,202006,제주특별자치도,제주시,영세,단란주점,00시,1520000,0,8,0,877005.7447,1.479816e+06,126.17888426813494,33.302765922881875,126.17888426813494_33.302765922881875,https://dapi.kakao.com/v2/local/geo/coord2regi...
2,2,3,3,202006,제주특별자치도,제주시,중소1,편의점,00시,482310,0,35,0,877056.6756,1.479616e+06,126.17945834265305,33.30097125427593,126.17945834265305_33.30097125427593,https://dapi.kakao.com/v2/local/geo/coord2regi...
3,3,4,4,202006,제주특별자치도,제주시,영세,편의점,00시,38050,5450,3,1,877055.9593,1.479766e+06,126.17943027296923,33.30232314934195,126.17943027296923_33.30232314934195,https://dapi.kakao.com/v2/local/geo/coord2regi...
4,4,5,5,202006,제주특별자치도,제주시,영세,일반한식,00시,32000,32000,1,1,877055.4817,1.479866e+06,126.17941155823847,33.30322441283756,126.17941155823847_33.30322441283756,https://dapi.kakao.com/v2/local/geo/coord2regi...


In [12]:
# 4. API를 통해 주소 가져오기 

import requests
import json

dong_ls = []

for i in range(len(df_uniq.index)):
    url = df_uniq['url'][i]
    headers = {"Authorization": "KakaoAK 9c01f59434bc3920843d9dd0279e6e6c"}
    api_test = requests.get(url, headers=headers)
    url_text = json.loads(api_test.text)
    dong = url_text['documents'][0]['region_3depth_name']
    dong_ls.append(dong)
print('converted point to addr')

converted point to addr


In [13]:
# 가져온 주소값 df 변환 
dic = {'dong':dong_ls}
df_dong = pd.DataFrame(dic)
df_dong.tail(3)

,dong
13906,남원읍
13907,구좌읍
13908,성산읍


In [14]:
# 5. df 와 merge 
df_join = df_uniq.join(df_dong)
df_join

,index,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lon,lat,lon_lat,url,dong
0,0,1,1,202006,제주특별자치도,제주시,영세,일반한식,00시,502000,0,10,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,https://dapi.kakao.com/v2/local/geo/coord2regi...,한경면
1,1,2,2,202006,제주특별자치도,제주시,영세,단란주점,00시,1520000,0,8,0,877005.7447,1.479816e+06,126.17888426813494,33.302765922881875,126.17888426813494_33.302765922881875,https://dapi.kakao.com/v2/local/geo/coord2regi...,한경면
2,2,3,3,202006,제주특별자치도,제주시,중소1,편의점,00시,482310,0,35,0,877056.6756,1.479616e+06,126.17945834265305,33.30097125427593,126.17945834265305_33.30097125427593,https://dapi.kakao.com/v2/local/geo/coord2regi...,한경면
3,3,4,4,202006,제주특별자치도,제주시,영세,편의점,00시,38050,5450,3,1,877055.9593,1.479766e+06,126.17943027296923,33.30232314934195,126.17943027296923_33.30232314934195,https://dapi.kakao.com/v2/local/geo/coord2regi...,한경면
4,4,5,5,202006,제주특별자치도,제주시,영세,일반한식,00시,32000,32000,1,1,877055.4817,1.479866e+06,126.17941155823847,33.30322441283756,126.17941155823847_33.30322441283756,https://dapi.kakao.com/v2/local/geo/coord2regi...,한경면
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13904,280449,280450,280450,202006,제주특별자치도,제주시,영세,기타숙박업,x시,200000,0,1,0,922462.7608,1.505523e+06,126.66487581142371,33.53884220315995,126.66487581142371_33.53884220315995,https://dapi.kakao.com/v2/local/geo/coord2regi...,조천읍
13905,280697,280698,280698,202006,제주특별자치도,서귀포시,영세,기타교육,x시,200000,0,2,0,925937.5363,1.479200e+06,126.70446614522854,33.30168772558733,126.70446614522854_33.30168772558733,https://dapi.kakao.com/v2/local/geo/coord2regi...,남원읍
13906,280815,280816,280816,202006,제주특별자치도,서귀포시,영세,제과점,x시,120000,0,1,0,927248.1999,1.476857e+06,126.71873169054687,33.28064712955715,126.71873169054687_33.28064712955715,https://dapi.kakao.com/v2/local/geo/coord2regi...,남원읍
13907,281123,281124,281124,202006,제주특별자치도,제주시,영세,학습지교육,x시,300000,0,1,0,937249.0613,1.507143e+06,126.82400884307276,33.554428760055124,126.82400884307276_33.554428760055124,https://dapi.kakao.com/v2/local/geo/coord2regi...,구좌읍


In [15]:
# 변환된 주소값을 원본 데이터와 합치기 

df_addr = df_join[['lon_lat','dong']]
df_addr.tail(3)

,lon_lat,dong
13906,126.71873169054687_33.28064712955715,남원읍
13907,126.82400884307276_33.554428760055124,구좌읍
13908,126.8835920054294_33.442727525516965,성산읍


In [17]:
df_final = pd.merge(df_6, df_addr, on='lon_lat')
df_final

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lon,lat,lon_lat,dong
0,1,1,202006,제주특별자치도,제주시,영세,일반한식,00시,502000,0,10,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
1,4972,4972,202006,제주특별자치도,제주시,영세,일반한식,01시,174000,0,6,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
2,19447,19447,202006,제주특별자치도,제주시,중소,한의원,07시,9600,0,4,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
3,23717,23717,202006,제주특별자치도,제주시,영세,의원,08시,53400,3400,2,1,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
4,23718,23718,202006,제주특별자치도,제주시,중소,한의원,08시,24000,0,10,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281891,280450,280450,202006,제주특별자치도,제주시,영세,기타숙박업,x시,200000,0,1,0,922462.7608,1.505523e+06,126.66487581142371,33.53884220315995,126.66487581142371_33.53884220315995,조천읍
281892,280698,280698,202006,제주특별자치도,서귀포시,영세,기타교육,x시,200000,0,2,0,925937.5363,1.479200e+06,126.70446614522854,33.30168772558733,126.70446614522854_33.30168772558733,남원읍
281893,280816,280816,202006,제주특별자치도,서귀포시,영세,제과점,x시,120000,0,1,0,927248.1999,1.476857e+06,126.71873169054687,33.28064712955715,126.71873169054687_33.28064712955715,남원읍
281894,281124,281124,202006,제주특별자치도,제주시,영세,학습지교육,x시,300000,0,1,0,937249.0613,1.507143e+06,126.82400884307276,33.554428760055124,126.82400884307276_33.554428760055124,구좌읍


In [18]:
# # 피클로 저장하기 

# import pickle
# with open('jeju_uniq_data6.pickle', 'wb') as fw:
#     pickle.dump(df_final, fw)


In [20]:
# import pickle
# with open('jeju_uniq_data6.pickle', 'rb') as fr:
#     df_pickle6 = pickle.load(fr)

# df_pickle6

In [ ]:
############# data_7월 ############## 

In [21]:
# 데이터 가져오기

raw_data_7 = pd.read_csv('../../1. raw_data/KRI-DAC_Jeju_data7.txt', sep=',')
raw_data_7.tail()

,OBJECTID,Field1,YM,SIDO,SIGUNGU,X,Y,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y
284260,284261,284261,202007,제주특별자치도,제주시,304250,100650,영세,기념품점,x시,6000,0,1,0,950524.6587,1.500760e+06
284261,284262,284262,202007,제주특별자치도,제주시,304250,100650,중소,농축수산품,x시,20000,0,1,0,950524.6587,1.500760e+06
284262,284263,284263,202007,제주특별자치도,제주시,304250,100650,중소,제과점,x시,30000,0,2,0,950524.6587,1.500760e+06
284263,284264,284264,202007,제주특별자치도,제주시,304300,100650,중소1,중국음식,x시,13000,0,1,0,950574.6388,1.500760e+06
284264,284265,284265,202007,제주특별자치도,제주시,304400,100650,영세,스넥,x시,9000,0,1,0,950674.5989,1.500760e+06


In [22]:
# 1. 좌표 변환 하기 

df_7 = raw_data_7.copy()

from pyproj import Proj, transform
# ITRF좌표계
proj_ITRF = Proj(init='epsg:5179')
# WGS 좌표계 
proj_WGS84 = Proj(init='epsg:4326')


df_7['lon'], df_7['lat'] = transform(proj_ITRF, proj_WGS84, df_7['POINT_X'], df_7['POINT_Y'])

df_7.tail(3)

,OBJECTID,Field1,YM,SIDO,SIGUNGU,X,Y,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lon,lat
284262,284263,284263,202007,제주특별자치도,제주시,304250,100650,중소,제과점,x시,30000,0,2,0,950524.6587,1.500760e+06,126.967369,33.497551
284263,284264,284264,202007,제주특별자치도,제주시,304300,100650,중소1,중국음식,x시,13000,0,1,0,950574.6388,1.500760e+06,126.967907,33.497555
284264,284265,284265,202007,제주특별자치도,제주시,304400,100650,영세,스넥,x시,9000,0,1,0,950674.5989,1.500760e+06,126.968983,33.497564


In [23]:
# 2. 좌표 구분 컬럼 생성 

df_7['lon'] = df_7['lon'].astype(str)
df_7['lat'] = df_7['lat'].astype(str)
df_7['lon_lat'] = df_7['lon']+'_'+df_7['lat']
df_7['lon_lat'].value_counts()

126.52427851055405_33.51290574138535     383
126.47793511858133_33.48574932701181     322
126.52935695016222_33.49899169591379     321
126.52880376641139_33.499886709171996    299
126.4902745974469_33.48770326836304      275
                                        ... 
126.57061364973353_33.24657586777765       1
126.65364028787246_33.46479009441826       1
126.33127404051857_33.4189514899592        1
126.55549055901162_33.25271259827109       1
126.45728611233939_33.46701213491232       1
Name: lon_lat, Length: 14000, dtype: int64

In [24]:
# 3. 중복 좌표 삭제 > url 생성 

df_uniq = df_7.drop_duplicates('lon_lat')
df_uniq.value_counts('lon_lat')

lon_lat
126.96951006044938_33.49847009919258     1
126.48537105406359_33.49124975974724     1
126.48551997546114_33.48268667698335     1
126.4855121390691_33.4831373658612       1
126.48549646585408_33.484038743513885    1
                                        ..
126.55813024766854_33.51600873041773     1
126.55812977546134_33.48355190164557     1
126.55811360750707_33.256349457237214    1
126.55804557742673_33.42359553568285     1
126.16370558240443_33.33544861735468     1
Length: 14000, dtype: int64

In [25]:
# url 생성

df_uniq['url1'] = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=' + df_uniq['lon']
df_uniq['url2'] = '&y=' + df_uniq['lat']
df_uniq['url'] = df_uniq['url1'] + df_uniq['url2']


df_uniq.drop(columns=['url1','url2'],axis=1,inplace=True)
df_uniq = df_uniq.reset_index()
df_uniq.head(5)

,index,OBJECTID,Field1,YM,SIDO,SIGUNGU,X,Y,FranClass,Type,...,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lon,lat,lon_lat,url
0,0,1,1,202007,제주특별자치도,제주시,230600,80000,영세,일반한식,...,85500,0,4,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,https://dapi.kakao.com/v2/local/geo/coord2regi...
1,1,2,2,202007,제주특별자치도,제주시,230600,80050,영세,단란주점,...,1960000,0,4,0,877005.7447,1.479816e+06,126.17888426813494,33.302765922881875,126.17888426813494_33.302765922881875,https://dapi.kakao.com/v2/local/geo/coord2regi...
2,2,3,3,202007,제주특별자치도,제주시,230650,79850,중소1,편의점,...,475650,0,43,0,877056.6756,1.479616e+06,126.17945834265305,33.30097125427593,126.17945834265305_33.30097125427593,https://dapi.kakao.com/v2/local/geo/coord2regi...
3,3,4,4,202007,제주특별자치도,제주시,230650,80000,영세,편의점,...,15650,0,1,0,877055.9593,1.479766e+06,126.17943027296923,33.30232314934195,126.17943027296923_33.30232314934195,https://dapi.kakao.com/v2/local/geo/coord2regi...
4,4,5,5,202007,제주특별자치도,제주시,230650,80100,영세,주점,...,82500,0,1,0,877055.4817,1.479866e+06,126.17941155823847,33.30322441283756,126.17941155823847_33.30322441283756,https://dapi.kakao.com/v2/local/geo/coord2regi...


In [26]:
# 4. API를 통해 주소 가져오기 

import requests
import json

dong_ls = []

for i in range(len(df_uniq.index)):
    url = df_uniq['url'][i]
    headers = {"Authorization": "KakaoAK 9c01f59434bc3920843d9dd0279e6e6c"}
    api_test = requests.get(url, headers=headers)
    url_text = json.loads(api_test.text)
    dong = url_text['documents'][0]['region_3depth_name']
    dong_ls.append(dong)
print('converted point to addr')

converted point to addr


In [27]:
# 가져온 주소값 df 변환 
dic = {'dong':dong_ls}
df_dong = pd.DataFrame(dic)
df_dong.tail(3)

,dong
13997,성산읍
13998,구좌읍
13999,성산읍


In [28]:
# 5. df 와 merge 
df_join = df_uniq.join(df_dong)
df_join

,index,OBJECTID,Field1,YM,SIDO,SIGUNGU,X,Y,FranClass,Type,...,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lon,lat,lon_lat,url,dong
0,0,1,1,202007,제주특별자치도,제주시,230600,80000,영세,일반한식,...,0,4,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,https://dapi.kakao.com/v2/local/geo/coord2regi...,한경면
1,1,2,2,202007,제주특별자치도,제주시,230600,80050,영세,단란주점,...,0,4,0,877005.7447,1.479816e+06,126.17888426813494,33.302765922881875,126.17888426813494_33.302765922881875,https://dapi.kakao.com/v2/local/geo/coord2regi...,한경면
2,2,3,3,202007,제주특별자치도,제주시,230650,79850,중소1,편의점,...,0,43,0,877056.6756,1.479616e+06,126.17945834265305,33.30097125427593,126.17945834265305_33.30097125427593,https://dapi.kakao.com/v2/local/geo/coord2regi...,한경면
3,3,4,4,202007,제주특별자치도,제주시,230650,80000,영세,편의점,...,0,1,0,877055.9593,1.479766e+06,126.17943027296923,33.30232314934195,126.17943027296923_33.30232314934195,https://dapi.kakao.com/v2/local/geo/coord2regi...,한경면
4,4,5,5,202007,제주특별자치도,제주시,230650,80100,영세,주점,...,0,1,0,877055.4817,1.479866e+06,126.17941155823847,33.30322441283756,126.17941155823847_33.30322441283756,https://dapi.kakao.com/v2/local/geo/coord2regi...,한경면
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13995,283079,283080,283080,202007,제주특별자치도,서귀포시,281050,77050,영세,일반한식,...,0,1,0,927447.1604,1.477058e+06,126.72085202231753,33.282472206838406,126.72085202231753_33.282472206838406,https://dapi.kakao.com/v2/local/geo/coord2regi...,남원읍
13996,283560,283561,283561,202007,제주특별자치도,서귀포시,291950,80650,영세,기타숙박업,...,0,1,0,938325.4075,1.480709e+06,126.8374176866993,33.316074929565715,126.8374176866993_33.316074929565715,https://dapi.kakao.com/v2/local/geo/coord2regi...,표선면
13997,283563,283564,283564,202007,제주특별자치도,서귀포시,292000,85450,영세,헬스크럽,...,0,1,0,938352.4466,1.485507e+06,126.83738053024116,33.35935333013854,126.83738053024116_33.35935333013854,https://dapi.kakao.com/v2/local/geo/coord2regi...,성산읍
13998,283696,283697,283697,202007,제주특별자치도,제주시,293800,104100,중소1,농축수산품,...,0,1,0,940062.3222,1.504158e+06,126.8545134611495,33.527665967051576,126.8545134611495_33.527665967051576,https://dapi.kakao.com/v2/local/geo/coord2regi...,구좌읍


In [29]:
# 변환된 주소값을 원본 데이터와 합치기 

df_addr = df_join[['lon_lat','dong']]
df_addr.tail(3)

,lon_lat,dong
13997,126.83738053024116_33.35935333013854,성산읍
13998,126.8545134611495_33.527665967051576,구좌읍
13999,126.9125257022098_33.45109898861301,성산읍


In [33]:
df_final = pd.merge(df_7, df_addr, on='lon_lat')
df_final

,OBJECTID,Field1,YM,SIDO,SIGUNGU,X,Y,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lon,lat,lon_lat,dong
0,1,1,202007,제주특별자치도,제주시,230600,80000,영세,일반한식,00시,85500,0,4,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
1,5076,5076,202007,제주특별자치도,제주시,230600,80000,영세,일반한식,01시,226000,0,4,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
2,19832,19832,202007,제주특별자치도,제주시,230600,80000,영세,한의원,07시,9600,0,4,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
3,24086,24086,202007,제주특별자치도,제주시,230600,80000,영세,한의원,08시,14400,0,6,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
4,30957,30957,202007,제주특별자치도,제주시,230600,80000,영세,한의원,09시,41900,0,9,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284260,283080,283080,202007,제주특별자치도,서귀포시,281050,77050,영세,일반한식,x시,210000,0,1,0,927447.1604,1.477058e+06,126.72085202231753,33.282472206838406,126.72085202231753_33.282472206838406,남원읍
284261,283561,283561,202007,제주특별자치도,서귀포시,291950,80650,영세,기타숙박업,x시,420000,0,1,0,938325.4075,1.480709e+06,126.8374176866993,33.316074929565715,126.8374176866993_33.316074929565715,표선면
284262,283564,283564,202007,제주특별자치도,서귀포시,292000,85450,영세,헬스크럽,x시,180000,0,1,0,938352.4466,1.485507e+06,126.83738053024116,33.35935333013854,126.83738053024116_33.35935333013854,성산읍
284263,283697,283697,202007,제주특별자치도,제주시,293800,104100,중소1,농축수산품,x시,3000,0,1,0,940062.3222,1.504158e+06,126.8545134611495,33.527665967051576,126.8545134611495_33.527665967051576,구좌읍


In [34]:
# # 피클로 저장하기 

# import pickle
# with open('jeju_uniq_data7.pickle', 'wb') as fw:
#     pickle.dump(df_final, fw)


In [36]:
# import pickle
# with open('jeju_uniq_data7.pickle', 'rb') as fr:
#     df_pickle7 = pickle.load(fr)

# df_pickle7

In [37]:
############# data 8월 ###########

In [38]:
# 데이터 가져오기

raw_data_8 = pd.read_csv('../../1. raw_data/KRI-DAC_Jeju_data8.txt', sep=',')
raw_data_8.tail()

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y
280080,280081,280081,202008,제주특별자치도,제주시,영세,편의점,x시,13700,0,2,0,950266.5976,1.502458e+06
280081,280082,280082,202008,제주특별자치도,제주시,영세,일반한식,x시,56000,0,1,0,950317.5379,1.502258e+06
280082,280083,280083,202008,제주특별자치도,제주시,영세,일반한식,x시,28000,0,1,0,950469.1585,1.501909e+06
280083,280084,280084,202008,제주특별자치도,제주시,중소1,중국음식,x시,46000,0,3,0,950574.6388,1.500760e+06
280084,280085,280085,202008,제주특별자치도,제주시,영세,서양음식,x시,9000,0,1,0,950565.5176,1.502659e+06


In [39]:
# 1. 좌표 변환 하기 

df_8 = raw_data_8.copy()

from pyproj import Proj, transform
# ITRF좌표계
proj_ITRF = Proj(init='epsg:5179')
# WGS 좌표계 
proj_WGS84 = Proj(init='epsg:4326')


df_8['lon'], df_8['lat'] = transform(proj_ITRF, proj_WGS84, df_8['POINT_X'], df_8['POINT_Y'])

df_8.tail(3)

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lon,lat
280082,280083,280083,202008,제주특별자치도,제주시,영세,일반한식,x시,28000,0,1,0,950469.1585,1.501909e+06,126.966708,33.507914
280083,280084,280084,202008,제주특별자치도,제주시,중소1,중국음식,x시,46000,0,3,0,950574.6388,1.500760e+06,126.967907,33.497555
280084,280085,280085,202008,제주특별자치도,제주시,영세,서양음식,x시,9000,0,1,0,950565.5176,1.502659e+06,126.967704,33.514684


In [40]:
# 2. 좌표 구분 컬럼 생성 

df_8['lon'] = df_8['lon'].astype(str)
df_8['lat'] = df_8['lat'].astype(str)
df_8['lon_lat'] = df_8['lon']+'_'+df_8['lat']
df_8['lon_lat'].value_counts()

126.52427851055405_33.51290574138535     366
126.52935695016222_33.49899169591379     302
126.47793511858133_33.48574932701181     301
126.52880376641139_33.499886709171996    292
126.57376368512601_33.51438638164462     271
                                        ... 
126.25738732521178_33.35032065718666       1
126.59183881308377_33.52811667401374       1
126.94623819475633_33.509545034887644      1
126.64155603093386_33.54985866185514       1
126.47206615420284_33.48297247109361       1
Name: lon_lat, Length: 13998, dtype: int64

In [41]:
# 3. 중복 좌표 삭제 > url 생성 

df_uniq = df_8.drop_duplicates('lon_lat')
df_uniq.value_counts('lon_lat')

lon_lat
126.96951006044938_33.49847009919258    1
126.48510753973808_33.47546908389631    1
126.48539457164357_33.48989769487357    1
126.48538673295272_33.49034838320207    1
126.48537105406359_33.49124975974724    1
                                       ..
126.55818756465061_33.25184225334594    1
126.55818016997517_33.25229297326027    1
126.55817277408272_33.25274369403429    1
126.55816759430778_33.51375522420132    1
126.16370558240443_33.33544861735468    1
Length: 13998, dtype: int64

In [42]:
# url 생성

df_uniq['url1'] = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=' + df_uniq['lon']
df_uniq['url2'] = '&y=' + df_uniq['lat']
df_uniq['url'] = df_uniq['url1'] + df_uniq['url2']


df_uniq.drop(columns=['url1','url2'],axis=1,inplace=True)
df_uniq = df_uniq.reset_index()
df_uniq.head(5)

,index,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lon,lat,lon_lat,url
0,0,1,1,202008,제주특별자치도,제주시,영세,일반한식,00시,249500,0,7,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,https://dapi.kakao.com/v2/local/geo/coord2regi...
1,1,2,2,202008,제주특별자치도,제주시,영세,단란주점,00시,2010000,0,7,0,877005.7447,1.479816e+06,126.17888426813494,33.302765922881875,126.17888426813494_33.302765922881875,https://dapi.kakao.com/v2/local/geo/coord2regi...
2,2,3,3,202008,제주특별자치도,제주시,중소1,편의점,00시,401210,0,41,0,877056.6756,1.479616e+06,126.17945834265305,33.30097125427593,126.17945834265305_33.30097125427593,https://dapi.kakao.com/v2/local/geo/coord2regi...
3,3,4,4,202008,제주특별자치도,제주시,영세,편의점,00시,12250,0,1,0,877055.9593,1.479766e+06,126.17943027296923,33.30232314934195,126.17943027296923_33.30232314934195,https://dapi.kakao.com/v2/local/geo/coord2regi...
4,4,5,5,202008,제주특별자치도,제주시,영세,주점,00시,33000,0,1,0,877055.4817,1.479866e+06,126.17941155823847,33.30322441283756,126.17941155823847_33.30322441283756,https://dapi.kakao.com/v2/local/geo/coord2regi...


In [43]:
# 4. API를 통해 주소 가져오기 

import requests
import json

dong_ls = []

for i in range(len(df_uniq.index)):
    url = df_uniq['url'][i]
    headers = {"Authorization": "KakaoAK 9c01f59434bc3920843d9dd0279e6e6c"}
    api_test = requests.get(url, headers=headers)
    url_text = json.loads(api_test.text)
    dong = url_text['documents'][0]['region_3depth_name']
    dong_ls.append(dong)
print('converted point to addr')

converted point to addr


In [44]:
# 가져온 주소값 df 변환 
dic = {'dong':dong_ls}
df_dong = pd.DataFrame(dic)
df_dong.tail(3)

,dong
13995,남원읍
13996,성산읍
13997,구좌읍


In [45]:
# 5. df 와 merge 
df_join = df_uniq.join(df_dong)
df_join

,index,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lon,lat,lon_lat,url,dong
0,0,1,1,202008,제주특별자치도,제주시,영세,일반한식,00시,249500,0,7,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,https://dapi.kakao.com/v2/local/geo/coord2regi...,한경면
1,1,2,2,202008,제주특별자치도,제주시,영세,단란주점,00시,2010000,0,7,0,877005.7447,1.479816e+06,126.17888426813494,33.302765922881875,126.17888426813494_33.302765922881875,https://dapi.kakao.com/v2/local/geo/coord2regi...,한경면
2,2,3,3,202008,제주특별자치도,제주시,중소1,편의점,00시,401210,0,41,0,877056.6756,1.479616e+06,126.17945834265305,33.30097125427593,126.17945834265305_33.30097125427593,https://dapi.kakao.com/v2/local/geo/coord2regi...,한경면
3,3,4,4,202008,제주특별자치도,제주시,영세,편의점,00시,12250,0,1,0,877055.9593,1.479766e+06,126.17943027296923,33.30232314934195,126.17943027296923_33.30232314934195,https://dapi.kakao.com/v2/local/geo/coord2regi...,한경면
4,4,5,5,202008,제주특별자치도,제주시,영세,주점,00시,33000,0,1,0,877055.4817,1.479866e+06,126.17941155823847,33.30322441283756,126.17941155823847_33.30322441283756,https://dapi.kakao.com/v2/local/geo/coord2regi...,한경면
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13993,275531,275532,275532,202008,제주특별자치도,제주시,영세,보습학원,x시,200000,0,1,0,910239.7342,1.500966e+06,126.53369851534627,33.49678916515772,126.53369851534627_33.49678916515772,https://dapi.kakao.com/v2/local/geo/coord2regi...,이도이동
13994,275663,275664,275664,202008,제주특별자치도,제주시,영세,학원(회원제형태),x시,220000,0,1,0,910390.6274,1.500767e+06,126.535342658025,33.4950054416331,126.535342658025_33.4950054416331,https://dapi.kakao.com/v2/local/geo/coord2regi...,이도이동
13995,278934,278935,278935,202008,제주특별자치도,서귀포시,영세,예체능학원,x시,110000,0,1,0,921003.9820,1.476178e+06,126.65174159048293,33.27407883370635,126.65174159048293_33.27407883370635,https://dapi.kakao.com/v2/local/geo/coord2regi...,남원읍
13996,279537,279538,279538,202008,제주특별자치도,서귀포시,영세,기타가구,x시,50000,0,1,0,937500.1674,1.486053e+06,126.82818252032293,33.36422597166203,126.82818252032293_33.36422597166203,https://dapi.kakao.com/v2/local/geo/coord2regi...,성산읍


In [46]:
# 변환된 주소값을 원본 데이터와 합치기 

df_addr = df_join[['lon_lat','dong']]
df_addr.tail(3)

,lon_lat,dong
13995,126.65174159048293_33.27407883370635,남원읍
13996,126.82818252032293_33.36422597166203,성산읍
13997,126.86586853032782_33.52371304945294,구좌읍


In [47]:
df_final = pd.merge(df_8, df_addr, on='lon_lat')
df_final

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lon,lat,lon_lat,dong
0,1,1,202008,제주특별자치도,제주시,영세,일반한식,00시,249500,0,7,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
1,5095,5095,202008,제주특별자치도,제주시,영세,일반한식,01시,109000,0,4,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
2,8991,8991,202008,제주특별자치도,제주시,영세,일반한식,02시,13500,0,1,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
3,20047,20047,202008,제주특별자치도,제주시,영세,한의원,07시,2400,0,1,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
4,24415,24415,202008,제주특별자치도,제주시,영세,의원,08시,18400,0,2,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280080,275532,275532,202008,제주특별자치도,제주시,영세,보습학원,x시,200000,0,1,0,910239.7342,1.500966e+06,126.53369851534627,33.49678916515772,126.53369851534627_33.49678916515772,이도이동
280081,275664,275664,202008,제주특별자치도,제주시,영세,학원(회원제형태),x시,220000,0,1,0,910390.6274,1.500767e+06,126.535342658025,33.4950054416331,126.535342658025_33.4950054416331,이도이동
280082,278935,278935,202008,제주특별자치도,서귀포시,영세,예체능학원,x시,110000,0,1,0,921003.9820,1.476178e+06,126.65174159048293,33.27407883370635,126.65174159048293_33.27407883370635,남원읍
280083,279538,279538,202008,제주특별자치도,서귀포시,영세,기타가구,x시,50000,0,1,0,937500.1674,1.486053e+06,126.82818252032293,33.36422597166203,126.82818252032293_33.36422597166203,성산읍


In [48]:
# # 피클로 저장하기 

# import pickle
# with open('jeju_uniq_data8.pickle', 'wb') as fw:
#     pickle.dump(df_final, fw)


In [49]:
import pickle
with open('jeju_uniq_data8.pickle', 'rb') as fr:
    df_pickle8 = pickle.load(fr)

df_pickle8

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lon,lat,lon_lat,dong
0,1,1,202008,제주특별자치도,제주시,영세,일반한식,00시,249500,0,7,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
1,5095,5095,202008,제주특별자치도,제주시,영세,일반한식,01시,109000,0,4,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
2,8991,8991,202008,제주특별자치도,제주시,영세,일반한식,02시,13500,0,1,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
3,20047,20047,202008,제주특별자치도,제주시,영세,한의원,07시,2400,0,1,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
4,24415,24415,202008,제주특별자치도,제주시,영세,의원,08시,18400,0,2,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280080,275532,275532,202008,제주특별자치도,제주시,영세,보습학원,x시,200000,0,1,0,910239.7342,1.500966e+06,126.53369851534627,33.49678916515772,126.53369851534627_33.49678916515772,이도이동
280081,275664,275664,202008,제주특별자치도,제주시,영세,학원(회원제형태),x시,220000,0,1,0,910390.6274,1.500767e+06,126.535342658025,33.4950054416331,126.535342658025_33.4950054416331,이도이동
280082,278935,278935,202008,제주특별자치도,서귀포시,영세,예체능학원,x시,110000,0,1,0,921003.9820,1.476178e+06,126.65174159048293,33.27407883370635,126.65174159048293_33.27407883370635,남원읍
280083,279538,279538,202008,제주특별자치도,서귀포시,영세,기타가구,x시,50000,0,1,0,937500.1674,1.486053e+06,126.82818252032293,33.36422597166203,126.82818252032293_33.36422597166203,성산읍


In [50]:
############# data_5월 ############## 

In [51]:
# 데이터 가져오기

raw_data_5 = pd.read_csv('../../1. raw_data/KRI-DAC_Jeju_data5.txt', sep=',')
raw_data_5.tail()

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y
273178,273179,273179,202005,제주특별자치도,제주시,중소,침구수예점,x시,17000,0,1,0,949616.6171,1.502505e+06
273179,273180,273180,202005,제주특별자치도,제주시,중소1,서양음식,x시,12000,0,1,0,949665.1566,1.502805e+06
273180,273181,273181,202005,제주특별자치도,제주시,영세,편의점,x시,4800,0,1,0,950266.5976,1.502458e+06
273181,273182,273182,202005,제주특별자치도,제주시,중소,제과점,x시,25000,0,2,0,950524.6587,1.500760e+06
273182,273183,273183,202005,제주특별자치도,제주시,영세,서양음식,x시,6000,0,1,0,950674.5989,1.500760e+06


In [52]:
# 1. 좌표 변환 하기 

df_5 = raw_data_5.copy()

from pyproj import Proj, transform
# ITRF좌표계
proj_ITRF = Proj(init='epsg:5179')
# WGS 좌표계 
proj_WGS84 = Proj(init='epsg:4326')


df_5['lon'], df_5['lat'] = transform(proj_ITRF, proj_WGS84, df_5['POINT_X'], df_5['POINT_Y'])

df_5.tail(3)

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lon,lat
273180,273181,273181,202005,제주특별자치도,제주시,영세,편의점,x시,4800,0,1,0,950266.5976,1.502458e+06,126.964496,33.512854
273181,273182,273182,202005,제주특별자치도,제주시,중소,제과점,x시,25000,0,2,0,950524.6587,1.500760e+06,126.967369,33.497551
273182,273183,273183,202005,제주특별자치도,제주시,영세,서양음식,x시,6000,0,1,0,950674.5989,1.500760e+06,126.968983,33.497564


In [53]:
# 2. 좌표 구분 컬럼 생성 

df_5['lon'] = df_5['lon'].astype(str)
df_5['lat'] = df_5['lat'].astype(str)
df_5['lon_lat'] = df_5['lon']+'_'+df_5['lat']
df_5['lon_lat'].value_counts()

126.52427851055405_33.51290574138535     400
126.52935695016222_33.49899169591379     314
126.52880376641139_33.499886709171996    309
126.47793511858133_33.48574932701181     275
126.56362561835856_33.24739705179205     258
                                        ... 
126.66989914559713_33.52717552512508       1
126.49490448290277_33.250191572152616      1
126.53890390710366_33.50721871134647       1
126.45950461288548_33.49363587313679       1
126.66543479759551_33.53749583871604       1
Name: lon_lat, Length: 13820, dtype: int64

In [54]:
# 3. 중복 좌표 삭제 > url 생성 

df_uniq = df_5.drop_duplicates('lon_lat')
df_uniq.value_counts('lon_lat')

lon_lat
126.96951006044938_33.49847009919258     1
126.48544160292364_33.487193564199174    1
126.48541302178809_33.45789217335456     1
126.48541024967045_33.488996318122375    1
126.48539457164357_33.48989769487357     1
                                        ..
126.55718128617019_33.247773337040584    1
126.5571738871868_33.24822405800287      1
126.5571664880694_33.2486747789311       1
126.55715908882763_33.24912549892337     1
126.16370558240443_33.33544861735468     1
Length: 13820, dtype: int64

In [56]:
# url 생성

df_uniq['url1'] = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=' + df_uniq['lon']
df_uniq['url2'] = '&y=' + df_uniq['lat']
df_uniq['url'] = df_uniq['url1'] + df_uniq['url2']


df_uniq.drop(columns=['url1','url2'],axis=1,inplace=True)
df_uniq = df_uniq.reset_index()
df_uniq

,level_0,index,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lon,lat,lon_lat,url
0,0,0,1,1,202005,제주특별자치도,제주시,영세,일반한식,00시,363000,66500,10,2,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,https://dapi.kakao.com/v2/local/geo/coord2regi...
1,1,1,2,2,202005,제주특별자치도,제주시,영세,단란주점,00시,1180000,0,3,0,877005.7447,1.479816e+06,126.17888426813494,33.302765922881875,126.17888426813494_33.302765922881875,https://dapi.kakao.com/v2/local/geo/coord2regi...
2,2,2,3,3,202005,제주특별자치도,제주시,중소1,편의점,00시,157670,6850,20,2,877056.6756,1.479616e+06,126.17945834265305,33.30097125427593,126.17945834265305_33.30097125427593,https://dapi.kakao.com/v2/local/geo/coord2regi...
3,3,3,4,4,202005,제주특별자치도,제주시,영세,편의점,00시,46600,0,2,0,877055.9593,1.479766e+06,126.17943027296923,33.30232314934195,126.17943027296923_33.30232314934195,https://dapi.kakao.com/v2/local/geo/coord2regi...
4,4,4,5,5,202005,제주특별자치도,제주시,영세,주점,00시,66000,0,2,0,877055.4817,1.479866e+06,126.17941155823847,33.30322441283756,126.17941155823847_33.30322441283756,https://dapi.kakao.com/v2/local/geo/coord2regi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13815,13815,268814,268815,268815,202005,제주특별자치도,서귀포시,영세,기타숙박업,x시,50000,0,1,0,889087.2224,1.472077e+06,126.30957647197593,33.23428902334641,126.30957647197593_33.23428902334641,https://dapi.kakao.com/v2/local/geo/coord2regi...
13816,13816,269415,269416,269416,202005,제주특별자치도,제주시,일반,건강식품(회원제형태),x시,7662600,0,12,0,903092.2237,1.501082e+06,126.45675120309421,33.49720789225105,126.45675120309421_33.49720789225105,https://dapi.kakao.com/v2/local/geo/coord2regi...
13817,13817,269435,269436,269436,202005,제주특별자치도,서귀포시,영세,기타숙박업,x시,240000,0,1,0,903666.4287,1.475096e+06,126.46571021256327,33.2629083000071,126.46571021256327_33.2629083000071,https://dapi.kakao.com/v2/local/geo/coord2regi...
13818,13818,270507,270508,270508,202005,제주특별자치도,제주시,일반,기타음료식품,x시,90750,0,1,0,907738.4497,1.501454e+06,126.50672452857444,33.50097517749024,126.50672452857444_33.50097517749024,https://dapi.kakao.com/v2/local/geo/coord2regi...


In [57]:
# 4. API를 통해 주소 가져오기 

import requests
import json

dong_ls = []

for i in range(len(df_uniq.index)):
    url = df_uniq['url'][i]
    headers = {"Authorization": "KakaoAK 9c01f59434bc3920843d9dd0279e6e6c"}
    api_test = requests.get(url, headers=headers)
    url_text = json.loads(api_test.text)
    dong = url_text['documents'][0]['region_3depth_name']
    dong_ls.append(dong)
print('converted point to addr')

converted point to addr


In [58]:
# 가져온 주소값 df 변환 
dic = {'dong':dong_ls}
df_dong = pd.DataFrame(dic)
df_dong.tail(3)

,dong
13817,하원동
13818,용담이동
13819,서홍동


In [59]:
# 5. df 와 merge 
df_join = df_uniq.join(df_dong)
df_join

,level_0,index,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,...,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lon,lat,lon_lat,url,dong
0,0,0,1,1,202005,제주특별자치도,제주시,영세,일반한식,00시,...,66500,10,2,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,https://dapi.kakao.com/v2/local/geo/coord2regi...,한경면
1,1,1,2,2,202005,제주특별자치도,제주시,영세,단란주점,00시,...,0,3,0,877005.7447,1.479816e+06,126.17888426813494,33.302765922881875,126.17888426813494_33.302765922881875,https://dapi.kakao.com/v2/local/geo/coord2regi...,한경면
2,2,2,3,3,202005,제주특별자치도,제주시,중소1,편의점,00시,...,6850,20,2,877056.6756,1.479616e+06,126.17945834265305,33.30097125427593,126.17945834265305_33.30097125427593,https://dapi.kakao.com/v2/local/geo/coord2regi...,한경면
3,3,3,4,4,202005,제주특별자치도,제주시,영세,편의점,00시,...,0,2,0,877055.9593,1.479766e+06,126.17943027296923,33.30232314934195,126.17943027296923_33.30232314934195,https://dapi.kakao.com/v2/local/geo/coord2regi...,한경면
4,4,4,5,5,202005,제주특별자치도,제주시,영세,주점,00시,...,0,2,0,877055.4817,1.479866e+06,126.17941155823847,33.30322441283756,126.17941155823847_33.30322441283756,https://dapi.kakao.com/v2/local/geo/coord2regi...,한경면
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13815,13815,268814,268815,268815,202005,제주특별자치도,서귀포시,영세,기타숙박업,x시,...,0,1,0,889087.2224,1.472077e+06,126.30957647197593,33.23428902334641,126.30957647197593_33.23428902334641,https://dapi.kakao.com/v2/local/geo/coord2regi...,안덕면
13816,13816,269415,269416,269416,202005,제주특별자치도,제주시,일반,건강식품(회원제형태),x시,...,0,12,0,903092.2237,1.501082e+06,126.45675120309421,33.49720789225105,126.45675120309421_33.49720789225105,https://dapi.kakao.com/v2/local/geo/coord2regi...,이호일동
13817,13817,269435,269436,269436,202005,제주특별자치도,서귀포시,영세,기타숙박업,x시,...,0,1,0,903666.4287,1.475096e+06,126.46571021256327,33.2629083000071,126.46571021256327_33.2629083000071,https://dapi.kakao.com/v2/local/geo/coord2regi...,하원동
13818,13818,270507,270508,270508,202005,제주특별자치도,제주시,일반,기타음료식품,x시,...,0,1,0,907738.4497,1.501454e+06,126.50672452857444,33.50097517749024,126.50672452857444_33.50097517749024,https://dapi.kakao.com/v2/local/geo/coord2regi...,용담이동


In [60]:
# 변환된 주소값을 원본 데이터와 합치기 

df_addr = df_join[['lon_lat','dong']]
df_addr.tail(3)

,lon_lat,dong
13817,126.46571021256327_33.2629083000071,하원동
13818,126.50672452857444_33.50097517749024,용담이동
13819,126.55548315030599_33.25316331771586,서홍동


In [62]:
df_final = pd.merge(df_5, df_addr, on='lon_lat')
df_final

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lon,lat,lon_lat,dong
0,1,1,202005,제주특별자치도,제주시,영세,일반한식,00시,363000,66500,10,2,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
1,4904,4904,202005,제주특별자치도,제주시,영세,일반한식,01시,86500,0,3,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
2,8569,8569,202005,제주특별자치도,제주시,영세,일반한식,02시,208000,0,5,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
3,11270,11270,202005,제주특별자치도,제주시,영세,일반한식,03시,18000,0,1,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
4,19122,19122,202005,제주특별자치도,제주시,중소,한의원,07시,4800,0,2,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273178,268815,268815,202005,제주특별자치도,서귀포시,영세,기타숙박업,x시,50000,0,1,0,889087.2224,1.472077e+06,126.30957647197593,33.23428902334641,126.30957647197593_33.23428902334641,안덕면
273179,269416,269416,202005,제주특별자치도,제주시,일반,건강식품(회원제형태),x시,7662600,0,12,0,903092.2237,1.501082e+06,126.45675120309421,33.49720789225105,126.45675120309421_33.49720789225105,이호일동
273180,269436,269436,202005,제주특별자치도,서귀포시,영세,기타숙박업,x시,240000,0,1,0,903666.4287,1.475096e+06,126.46571021256327,33.2629083000071,126.46571021256327_33.2629083000071,하원동
273181,270508,270508,202005,제주특별자치도,제주시,일반,기타음료식품,x시,90750,0,1,0,907738.4497,1.501454e+06,126.50672452857444,33.50097517749024,126.50672452857444_33.50097517749024,용담이동


In [63]:
# # 피클로 저장하기 

# import pickle
# with open('jeju_uniq_data5.pickle', 'wb') as fw:
#     pickle.dump(df_final, fw)


In [64]:
import pickle
with open('jeju_uniq_data5.pickle', 'rb') as fr:
    df_pickle5 = pickle.load(fr)

df_pickle5

,OBJECTID,Field1,YM,SIDO,SIGUNGU,FranClass,Type,Time,TotalSpent,DisSpent,NumofSpent,NumofDisSpent,POINT_X,POINT_Y,lon,lat,lon_lat,dong
0,1,1,202005,제주특별자치도,제주시,영세,일반한식,00시,363000,66500,10,2,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
1,4904,4904,202005,제주특별자치도,제주시,영세,일반한식,01시,86500,0,3,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
2,8569,8569,202005,제주특별자치도,제주시,영세,일반한식,02시,208000,0,5,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
3,11270,11270,202005,제주특별자치도,제주시,영세,일반한식,03시,18000,0,1,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
4,19122,19122,202005,제주특별자치도,제주시,중소,한의원,07시,4800,0,2,0,877005.9834,1.479766e+06,126.17889362710162,33.30231529076243,126.17889362710162_33.30231529076243,한경면
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
273178,268815,268815,202005,제주특별자치도,서귀포시,영세,기타숙박업,x시,50000,0,1,0,889087.2224,1.472077e+06,126.30957647197593,33.23428902334641,126.30957647197593_33.23428902334641,안덕면
273179,269416,269416,202005,제주특별자치도,제주시,일반,건강식품(회원제형태),x시,7662600,0,12,0,903092.2237,1.501082e+06,126.45675120309421,33.49720789225105,126.45675120309421_33.49720789225105,이호일동
273180,269436,269436,202005,제주특별자치도,서귀포시,영세,기타숙박업,x시,240000,0,1,0,903666.4287,1.475096e+06,126.46571021256327,33.2629083000071,126.46571021256327_33.2629083000071,하원동
273181,270508,270508,202005,제주특별자치도,제주시,일반,기타음료식품,x시,90750,0,1,0,907738.4497,1.501454e+06,126.50672452857444,33.50097517749024,126.50672452857444_33.50097517749024,용담이동


In [ ]:
# 1. 데이터 가져오기 (5~8월)

import pickle
with open('jeju_uniq_data5.pickle','rb') as fr:
    jeju_5 = pickle.load(fr)
with open('jeju_uniq_data6.pickle','rb') as fr:
    jeju_6 = pickle.load(fr)
with open('jeju_uniq_data7.pickle','rb') as fr:
    jeju_7 = pickle.load(fr)
with open('jeju_uniq_data8.pickle','rb') as fr:
    jeju_8 = pickle.load(fr)

In [ ]:
jeju_7.drop(columns=['X','Y'],axis=1,inplace=True)
jeju_all = pd.concat([jeju_5,jeju_6,jeju_7,jeju_8],axis=0, sort=False)
jeju_all.head()

In [ ]:
# # 피클 저장하기 

# with open('jeju_all.pickle','wb') as fw:
#     pickle.dump(jeju_all, fw)